# Training

In [1]:
import csv
import pandas as pd

train_commands = []

base_path = "/visinf/home/vilab25/Confidence-based-generative-data-Augmentation-for-Meta-Learning"
for mode in ["run_within_domain", "run_cross_domain"]:
    if "run_within_domain":
        mode_shortcut = "WD"
    else:
        mode_shortcut = "CD"
    

    for model in ["finetuning", "maml"]:
        
        for dropout_mode in ["baseline", "dropout"]:
            
            for k in [1, 5, 10]:
                cmd_strig = f"""{base_path}/scripts/isolated_run.sh \
                {base_path}/scripts/train/train.sh \
                --datasets_dir "/fastdata/vilab24/meta-album" \
                --{mode} \
                --model_dir "baselines/{model}" \
                --config_path "{base_path}/configs/train/{dropout_mode}/{mode_shortcut}/{dropout_mode}_k_{k}.yml" \
                --output_dir "/fastdata/vilab25/output/scheduler/training" \
                --log_file /visinf/home/vilab25/logs/train_{mode}_{model}_{dropout_mode}_k_{k}.log
                """
                
                train_commands.append(cmd_strig)
                
                

# Write commands to CSV file
csv_file_path = "scripts1.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["ID", "command", "status", "attached to process", "attached to gpu", "process alive", "expected_cpu_usage (in %)", "expected_gpu_usage (in MB)", "timestamp_started", "last_timestamp"])
    for index, cmd in enumerate(train_commands[::2], start=1):
        writer.writerow([index, cmd, "waiting", " ", " ", " ", "1800", "3500", " ", " "])

print(f"CSV file '{csv_file_path}' has been created successfully.")

# Write commands to CSV file
csv_file_path = "scripts2.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["ID", "command", "status", "attached to process", "attached to gpu", "process alive", "expected_cpu_usage (in %)", "expected_gpu_usage (in MB)", "timestamp_started", "last_timestamp"])
    for index, cmd in enumerate(train_commands[1::2], start=1):
        writer.writerow([index, cmd, "waiting", " ", " ", " ", "1800", "3500", " ", " "])

print(f"CSV file '{csv_file_path}' has been created successfully.")

CSV file 'scripts1.csv' has been created successfully.
CSV file 'scripts2.csv' has been created successfully.


/tmp/ipykernel_4074489/2670594554.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Eval

In [2]:
import itertools


dropout_mode = ["baseline", "dropout"]
confidence_method = ['"ConstantConfidenceProvider"', '"MCDropoutConfidenceEstimator"', '"GTConfidenceEstimator"']
data_aug = ['"null"', '"StandardAugmentation"', '"GenerativeAugmentation"', '"PseudoAugmentation"']

ignored_config = []
configs = []
for config in itertools.product(dropout_mode, confidence_method, data_aug):
    match config:
        case ("baseline", '"ConstantConfidenceProvider"', '"null"'): 
            configs.append(config)
        case ("baseline", _, _): 
            ignored_config.append(config)
        case ("dropout", '"ConstantConfidenceProvider"', _):
            configs.append(config)
        case ("dropout", _, '"null"'):
            ignored_config.append(config)
        case ("dropout", '"MCDropoutConfidenceEstimator"' | '"GTConfidenceEstimator"', _):
            configs.append(config)       
        case _:
            print(config)
            
            
model = ["finetuning", "maml"]
mode = ["run_within_domain", "run_cross_domain"]
k = [1, 5, 10]

base_product = itertools.product(model, mode, k)

full_config_dict = []

for model, mode, k in base_product:
    for dropout_mode, confidence_method, data_aug  in configs:
        full_config_dict.append({"model": model,
         "mode": mode,
         "k": k,
         "k_config": k + 10*(data_aug == '"PseudoAugmentation"'),
         "dropout_mode": dropout_mode,
         "confidence_method": confidence_method,
         "confidence": 1 if data_aug == '"null"' else 0,
         "data_aug": data_aug,
         })


In [3]:
def cmd_script_eval(base_path, config, config_path):
    k = config['k']
    mode = config['mode'] 
    dropout_mode = config['dropout_mode']
    model = config['model']
    return f"""{base_path}/scripts/isolated_run.sh \
            {base_path}/scripts/train/train.sh \
            --datasets_dir "/fastdata/vilab24/meta-album" \
            --ks "{k}" \
            --{mode} \
            --model_path "/fastdata/vilab25/output/scheduler/training/{dropout_mode}" \
            --model {model} \
            --config_path "{config_path}" \
            --output_dir "/fastdata/vilab25/output/scheduler/eval/" \
            --log_file /visinf/home/vilab25/logs/eval_{dropout_mode}_{k}_{model}_{mode}.log
            """

def generate_config(config):
    return f"""dataset: 
    test: 
        n_ways:
            value: 5
        k_shots: 
            value: {config["k_config"]}
        query_size: 20
            
    evaluation:
        tasks_per_dataset: 100
        confidence_estimators:
            use: {config["confidence_method"]}
            ConstantConfidenceProvider:
                confidence: {config["confidence"]}
            GTConfidenceEstimator: {{}} 
            MCDropoutConfidenceEstimator:
                num_samples: 100

        augmentors:
            use: {config["data_aug"]}
            PseudoAugmentation: 
                keep_original_data: True
                augmentation_size: 
                    scale: 1
                    offset: 3 
                    maximum: 20
                    threshold: 0.8
            StandardAugmentation:
                threshold: 0.75
                scale: 2
                keep_original_data: True
                rotation: 25
                flip: True
            GenerativeAugmentation:
                keep_original_data: True
                augmentation_size: 
                    scale: 1
                    offset: 3 
                    maximum: 20
                    threshold: 0.8
                annotator_type: "hed"
    """

def generate_config_path(config):
    dropout_mode = config["dropout_mode"]
    
    confidence_method =  {'"ConstantConfidenceProvider"': "constant_confidence",
                        '"MCDropoutConfidenceEstimator"': "mc", 
                        '"GTConfidenceEstimator"': "gt"}[config["confidence_method"]]
    
    data_aug = {'"null"': "baseline",
                '"StandardAugmentation"': "standard_aug",
                '"GenerativeAugmentation"': "generative_aug",
                '"PseudoAugmentation"': "pseudo_aug"}[config['data_aug']]

    k = config["k"]
    return f"configs/eval/{dropout_mode}_{data_aug}_{confidence_method}_k_{k}.yml"

In [4]:
all_paths = [generate_config_path(config) for config in full_config_dict]
all_configs = [generate_config(config) for config in full_config_dict]
base_path = "/visinf/home/vilab25/Confidence-based-generative-data-Augmentation-for-Meta-Learning"

cmd_scripts = [cmd_script_eval(base_path, config, config_path) for config, config_path in zip(full_config_dict, all_paths)]

In [5]:
df= pd.DataFrame(full_config_dict)
df["config_path"] = all_paths
df["config"] = all_configs
df["cmd_scripts"] = cmd_scripts

In [15]:
# Check for duplicates
print(df.groupby("cmd_scripts")["config"].value_counts().sort_values())

print(df.groupby("config_path")["config"].nunique().sort_values())

cmd_scripts                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  config                                                                                                                                                                                                                                                                                     

In [45]:
df

,model,mode,k,k_config,dropout_mode,confidence_method,confidence,data_aug,config_path,config,cmd_scripts
0,finetuning,run_within_domain,1,1,baseline,"""ConstantConfidenceProvider""",1,"""null""",configs/eval/baseline_baseline_constant_confid...,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
1,finetuning,run_within_domain,1,1,dropout,"""ConstantConfidenceProvider""",1,"""null""",configs/eval/dropout_baseline_constant_confide...,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
2,finetuning,run_within_domain,1,1,dropout,"""ConstantConfidenceProvider""",0,"""StandardAugmentation""",configs/eval/dropout_standard_aug_constant_con...,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
3,finetuning,run_within_domain,1,1,dropout,"""ConstantConfidenceProvider""",0,"""GenerativeAugmentation""",configs/eval/dropout_generative_aug_constant_c...,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
4,finetuning,run_within_domain,1,11,dropout,"""ConstantConfidenceProvider""",0,"""PseudoAugmentation""",configs/eval/dropout_pseudo_aug_constant_confi...,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
...,...,...,...,...,...,...,...,...,...,...,...
127,maml,run_cross_domain,10,10,dropout,"""MCDropoutConfidenceEstimator""",0,"""GenerativeAugmentation""",configs/eval/dropout_generative_aug_mc_k_10.yml,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
128,maml,run_cross_domain,10,20,dropout,"""MCDropoutConfidenceEstimator""",0,"""PseudoAugmentation""",configs/eval/dropout_pseudo_aug_mc_k_10.yml,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
129,maml,run_cross_domain,10,10,dropout,"""GTConfidenceEstimator""",0,"""StandardAugmentation""",configs/eval/dropout_standard_aug_gt_k_10.yml,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...
130,maml,run_cross_domain,10,10,dropout,"""GTConfidenceEstimator""",0,"""GenerativeAugmentation""",configs/eval/dropout_generative_aug_gt_k_10.yml,dataset: \n test: \n n_ways:\n ...,/visinf/home/vilab25/Confidence-based-generati...


In [46]:
# Anlegen von ymls
grouped = df.groupby('config_path')

for config_path, group_df in grouped:
    with open(base_path+"/"+config_path, 'w+') as f:
        for config in group_df['config']:
            f.write(config )